<a href="https://colab.research.google.com/github/financieras/math_for_ai/blob/main/estadistica/articulo2_evaluando_regresion_lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluando Regresión Lineal: Métricas, Validación y Cuándo Usarla

Ya sabes implementar regresión lineal por mínimos cuadrados. Pero un modelo que hace predicciones no es útil si no sabes **qué tan buenas son esas predicciones** y **cuándo es apropiado usarlo**.

En este artículo aprenderás a:
- Evaluar el rendimiento con métricas estándar (RMSE, R²)
- Interpretar los resultados correctamente
- Comparar tu implementación con scikit-learn
- Identificar las ventajas y limitaciones del método
- Decidir cuándo usar mínimos cuadrados vs otros enfoques

**Nota**: Este artículo asume que ya implementaste regresión lineal por mínimos cuadrados. Si no lo has hecho, comienza por el artículo anterior donde construimos el modelo desde cero.

---


## Configuración inicial

Primero recreamos el modelo que construimos en el artículo anterior:


In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Datos de ejemplo: tamaño de casa vs precio
superficie = np.array([50, 55, 60, 64, 70, 78, 80, 89, 90, 100])
precio = np.array([140000, 155000, 190000, 170000, 200000, 230000, 240000, 260000, 270000, 300000])

# Matriz de diseño y función de mínimos cuadrados
X = np.column_stack([np.ones(len(superficie)), superficie])
y = precio

def minimos_cuadrados(X, y):
    """Calcula los coeficientes óptimos usando mínimos cuadrados"""
    return np.linalg.inv(X.T @ X) @ X.T @ y

# Calcular coeficientes
w = minimos_cuadrados(X, y)
precio_predicho = X @ w

print(f"Coeficientes calculados:")
print(f"  w0 (intercept) = {w[0]:.2f} €")
print(f"  w1 (pendiente) = {w[1]:.2f} €/m²")


Coeficientes calculados:
  w0 (intercept) = -17518.73 €
  w1 (pendiente) = 3166.02 €/m²



---


## Evaluación del modelo

Para evaluar qué tan bien funciona nuestro modelo, utilizamos dos métricas fundamentales:

**RMSE (Root Mean Square Error)**: Es la raíz cuadrada del MSE y nos dice el error promedio en las mismas unidades que nuestra variable objetivo (euros). Un RMSE bajo indica predicciones más precisas.

$$\text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n}(y_i - \hat{y}_i)^2}$$

**R² (Coeficiente de Determinación)**: Indica qué proporción de la variabilidad de los datos es explicada por nuestro modelo. Varía entre 0 y 1, donde 1 significa ajuste perfecto.

$$R^2 = 1 - \frac{\sum_{i=1}^{n}(y_i - \hat{y}_i)^2}{\sum_{i=1}^{n}(y_i - \bar{y})^2}$$

### Conexión con $R^2$: del "cuánto se mueven juntos" al "cuánto explica el modelo"

Ahora viene la magia: **en regresión lineal simple (una sola variable), $R^2 = r^2$**.

Es decir:
- $R^2$ responde: *“¿Qué porcentaje de la variabilidad en $y$ explica mi modelo?”*
- $r^2$ responde lo mismo, **pero partiendo solo de la correlación**.

Por eso:
- Si $r = 0.5$, entonces $R^2 = 0.25$ → el modelo explica el **25%** de la variabilidad.
- Si $r = 0$, entonces $R^2 = 0$ → el modelo no explica nada.

> **En resumen**:  
> - Usa $r$ para entender la **relación cruda** entre $x$ e $y$ y ver el signo para saber si la relación es directa o inversa.  
> - Usa $R^2$ para saber **cuánto de esa relación captura tu modelo lineal**, y expresar esa relación en porcentaje.

In [2]:
# Calculamos el error cuadrático medio (MSE) y RMSE
mse = np.mean((precio - precio_predicho) ** 2)
rmse = np.sqrt(mse)

# Coeficiente de determinación R²
ss_res = np.sum((precio - precio_predicho) ** 2)  # Suma de cuadrados de residuos
ss_tot = np.sum((precio - precio.mean()) ** 2)    # Suma total de cuadrados
r2 = 1 - (ss_res / ss_tot)

# Métricas de rendimiento
print("\n" + "="*50)
print("MÉTRICAS DE RENDIMIENTO")
print("="*50)
print(f"R² (coeficiente de determinación): {r2:.3f}")
print(f"  → El modelo explica el {r2*100:.1f}% de la variabilidad")
print(f"\nRMSE (raiz del error cuadrático medio): {rmse:,.2f}€")
print(f"  → Error promedio de {rmse:,.0f} € en las predicciones")


MÉTRICAS DE RENDIMIENTO
R² (coeficiente de determinación): 0.976
  → El modelo explica el 97.6% de la variabilidad

RMSE (raiz del error cuadrático medio): 7,748.54€
  → Error promedio de 7,749 € en las predicciones


Un $R^2$ de 0.9 indica que nuestro modelo es excelente, explicando el 90% de la variabilidad en los precios. Sin embargo, el RMSE de ~14.5k€ nos recuerda que aún hay margen de error considerable en las predicciones individuales.

### Interpretación
- Un MSE más bajo es mejor, pero su valor está en euros al cuadrado, lo que lo hace difícil de interpretar.  
Por eso usamos el RMSE, que está en euros y representa el error promedio en las predicciones.
- El coeficiente de determinación $R^2$ es la métrica más intuitiva:
    - $0 \le R^2 \le 1$: El modelo explica un porcentaje de la variabilidad
    - $R^2 = 1$: Modelo perfecto (explica el 100% de la variabilidad)
    - $R^2 = 0.9$: El modelo explica el 90% de la variabilidad en los datos
    - $R^2 = 0$: El modelo no es mejor que simplemente predecir el promedio

## Haciendo predicciones

In [3]:
# Predicciones para nuevas casas
nuevas_superficies = np.array([65, 85, 95])
predicciones = w[0] + w[1] * nuevas_superficies

print("\nPREDICCIONES PARA NUEVAS VIVIENDAS")
print("-" * 34)
for sup, pred in zip(nuevas_superficies, predicciones):
    print(f"   Casa de {sup}m²  →  {pred:,.0f}€")


PREDICCIONES PARA NUEVAS VIVIENDAS
----------------------------------
   Casa de 65m²  →  188,272€
   Casa de 85m²  →  251,593€
   Casa de 95m²  →  283,253€


## Comparación con scikit-learn

In [4]:
from sklearn.linear_model import LinearRegression

# Creamos y entrenamos el modelo
modelo_sklearn = LinearRegression()
modelo_sklearn.fit(superficie.reshape(-1, 1), precio)

# Comparamos resultados
print("\n" + "="*60)
print("COMPARACIÓN: Nuestra Implementación vs scikit-learn")
print("="*60)
print(f"\n{'Parámetro':<25} {'Nuestra impl.':<20} {'scikit-learn':<20}")
print("-" * 60)
print(f"{'w₀ (intercepto)':<25} {w[0]:>15,.2f}€   {modelo_sklearn.intercept_:>15,.2f}€")
print(f"{'w₁ (pendiente)':<25} {w[1]:>15,.2f}€/m² {modelo_sklearn.coef_[0]:>14,.2f}€/m²")
print("\n¡Los resultados son idénticos! ✓")


COMPARACIÓN: Nuestra Implementación vs scikit-learn

Parámetro                 Nuestra impl.        scikit-learn        
------------------------------------------------------------
w₀ (intercepto)                -17,518.73€        -17,518.73€
w₁ (pendiente)                   3,166.02€/m²       3,166.02€/m²

¡Los resultados son idénticos! ✓


# **4. Ventajas, Limitaciones y Cuándo Usarlo**

## Ventajas del método de mínimos cuadrados

- **Solución exacta**: Encuentra los coeficientes óptimos directamente, sin iteraciones
- **Rápido y eficiente**: Para datasets pequeños y medianos, es computacionalmente muy eficiente
- **Interpretabilidad**: Los coeficientes tienen una interpretación directa y clara
- **Garantía matemática**: Si existe solución, este método la encuentra
- **Sin hiperparámetros**: No requiere ajuste de learning rate u otros parámetros

## Limitaciones importantes

In [5]:
# Ejemplo de problema con multicolinealidad
X_problema = np.column_stack([superficie, superficie * 2])  # Columnas linealmente dependientes
print("Matriz con columnas linealmente dependientes:")
print(X_problema[:3])
print(f"\nDeterminante de X.T @ X: {np.linalg.det(X_problema.T @ X_problema):.10f}")
print("Un determinante cercano a 0 indica problemas de inversión matricial")

Matriz con columnas linealmente dependientes:
[[ 50 100]
 [ 55 110]
 [ 60 120]]

Determinante de X.T @ X: 0.0000000000
Un determinante cercano a 0 indica problemas de inversión matricial


**Problemas comunes:**
- **Matrices singulares**: Cuando las variables son linealmente dependientes
- **Multicolinealidad**: Variables predictoras muy correlacionadas entre sí
- **No escala bien**: Para datasets muy grandes (millones de registros), la inversión de matriz se vuelve costosa ($O(n^3)$)
- **Sensibilidad a outliers**: Los errores al cuadrado amplifican el efecto de valores atípicos

Ahora que conocemos las limitaciones del método, surge la pregunta natural: ¿cuándo debemos usar mínimos cuadrados y cuándo recurrir a alternativas como Gradient Descent?

## ¿Cuándo usar mínimos cuadrados vs Gradient Descent?

**Usa mínimos cuadrados cuando:**
- Tienes < 10,000 muestras y < 100 características
- Necesitas la solución exacta en una sola operación
- El dataset cabe cómodamente en memoria RAM

**Usa Gradient Descent cuando:**
- Tienes millones de registros o cientos de características
- El dataset no cabe en memoria (puedes usar mini-batches)
- Necesitas actualizar el modelo con nuevos datos continuamente
- Trabajas con redes neuronales u otros modelos no lineales

---

# **5. Conclusión**

Hemos visto cómo el método de mínimos cuadrados nos permite encontrar la mejor recta que se ajusta a nuestros datos mediante una solución matemática elegante y directa.

**En resumen:**
- **Fácil de implementar** con pocas líneas de código
- **Resultados interpretables** que podemos explicar a cualquier stakeholder
- **Extremadamente efectivo** para problemas con relaciones lineales
- **Base fundamental** para entender métodos más complejos

**Próximos pasos:** En el siguiente artículo de esta serie, exploraremos el **algoritmo de Gradient Descent**, que nos permitirá escalar a problemas más grandes, allanando el camino hacia técnicas más avanzadas de Machine Learning.